In [1]:
import inspect
import os
import sys

currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0, parentdir)

import numpy as np
import matplotlib.pyplot as plt  # Gives access to basic plotting functions
from natsort import natsorted

import pandas as pd
import matplotlib.tri as mtri
import csv

In [2]:
# Create a grid for the interpolation.
xi = np.linspace(0.0955, 0.1065, 2000)
mesh_dx = xi[1]-xi[0] # linspace is equidistant
yi = np.linspace(0.0, 0.0105765, 1923)
mesh_dy = yi[1]-yi[0] # linspace is equidistant
Xi, Yi = np.meshgrid(xi, yi)

diffusion_coefficient = 1.32E-05 # in m2/s

# Create a dictionary to store data for each target number
data_by_target = {}

# Calculate time for x-axis 
x_axis_time = np.linspace(0.0, 23.60, 60)
# 
folder_path = 'fieldData'

# Initialize data arrays for this target number
target_mean_fluxes = []
target_mean_fluxes_error = []
# Loop through the range of values from 400 to 24000 with a step of 400
time_range = list(range(0, 24000, 400))
for value in time_range:
    # Construct the complete filename
    filename = os.path.join(folder_path, f'time_{value}.csv')
    
    # Read the CSV file into a DataFrame
    df = pd.read_csv(filename)#, delimiter=', ', engine='python')
    
    c_data = df['C'].values
    coords_0 = df['coords_0'].values
    coords_1 = df['coords_1'].values
    # create a triangulation of the (coords_0, coords_1) points
    triang = mtri.Triangulation(coords_0, coords_1)
    conc_interpolator = mtri.CubicTriInterpolator(triang, c_data, kind='geom')
    concentration_matrix = conc_interpolator(Xi, Yi)
    conc_grad_interpolation = conc_interpolator.gradient(Xi, Yi) # compute values for the gradient
    #######ax2.pcolormesh(concentration_matrix)
    masked_conc = concentration_matrix
    print (masked_conc.shape)
    ######### diffusion flux
    diffusion_flux_0_interpolation = - diffusion_coefficient * conc_grad_interpolation[0]
    diffusion_flux_1_interpolation = - diffusion_coefficient * conc_grad_interpolation[1]
    #########
    # grad_y, grad_x = np.gradient(masked_conc, mesh_dy, mesh_dx)#5.5E-6,5.5E-06)
    # diffusion_flux_1_interpolation = grad_y * (-1) * 1.32E-5
    # diffusion_flux_0_interpolation = grad_x * (-1) * 1.32E-5
    ##########
    horizontal_start = 818
    horizontal_end = 1364
    list_of_fluxes = []
    for pixels_above_edge in np.linspace(0.02E-3 / 5.5E-6, 1E-3 / 5.5E-6, 60, dtype=int):
        vertical_row =  pixels_above_edge #substrate_position +
        flux1 = np.trapz(diffusion_flux_1_interpolation[vertical_row, horizontal_start - pixels_above_edge:horizontal_end + pixels_above_edge],
                        dx=mesh_dy) #dx=5.5E-6
        
        flux2 = np.trapz(diffusion_flux_0_interpolation[0:vertical_row, horizontal_start - pixels_above_edge], dx=mesh_dx ) #dx=5.5E-6
    
        flux3 = np.trapz(diffusion_flux_0_interpolation[0:vertical_row, horizontal_end + pixels_above_edge], dx=mesh_dx) #dx=5.5E-6
        
        total_flux = flux1 + (flux3 - flux2)
        
        print('The overall diffusive flux along pixel_y for ' + str(value) + ' =' + str(pixels_above_edge) + ' is  ' + str(total_flux))
    
        list_of_fluxes.append(total_flux)
    mean_flux = np.mean(list_of_fluxes)
    target_mean_fluxes.append(mean_flux)
    print('The mean flux is ' + str(np.mean(list_of_fluxes)))
    error_of_mean = np.std(list_of_fluxes, ddof=1, keepdims=True) / (np.size(list_of_fluxes)) ** 0.5
    target_mean_fluxes_error.append(error_of_mean)
    print('The sample standard error of mean is ' + str(np.std(list_of_fluxes, ddof=1) / (np.size(list_of_fluxes)) ** 0.5))

target_mean_fluxes = [i*0.060 for i in target_mean_fluxes] 
# Specify the CSV file name
file_name = '0.960C_mesh3.csv'

# Combine the two lists into rows
rows = zip(x_axis_time, target_mean_fluxes)

# Write the rows into a CSV file
with open(file_name, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['time(s)', 'flux(mol/s)'])  # Optional: Write column headers
    writer.writerows(rows)


(1923, 2000)
The overall diffusive flux along pixel_y for 0 =3 is  0.00046613386808276987
The overall diffusive flux along pixel_y for 0 =6 is  0.00041878534887984637
The overall diffusive flux along pixel_y for 0 =9 is  0.000308924966525611
The overall diffusive flux along pixel_y for 0 =12 is  0.00026950384710952715
The overall diffusive flux along pixel_y for 0 =15 is  0.00023111719179229415
The overall diffusive flux along pixel_y for 0 =18 is  0.00019260446866012926
The overall diffusive flux along pixel_y for 0 =21 is  0.00015391769208584612
The overall diffusive flux along pixel_y for 0 =24 is  0.00011562862416967603
The overall diffusive flux along pixel_y for 0 =27 is  5.20932938323752e-05
The overall diffusive flux along pixel_y for 0 =30 is  4.366916305969212e-06
The overall diffusive flux along pixel_y for 0 =33 is  -2.3929508945511807e-05
The overall diffusive flux along pixel_y for 0 =36 is  -3.4363878282981705e-05
The overall diffusive flux along pixel_y for 0 =39 is  -2